<a href="https://colab.research.google.com/github/hammaad2002/ASRAdversarialAttacks/blob/main/Tutorial(How_to_use_this_repository).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone https://github.com/hammaad2002/ASRAdversarialAttacks.git
!git clone https://github.com/hammaad2002/CRDNN_Model.git
%cd /content/ASRAdversarialAttacks/
!pip install -r requirements.txt

In [2]:
from ASRAdversarialAttacks.AdversarialAttacks import ASRAttacks
import torchaudio
import torch
import numpy as np
from IPython.display import Audio

In [3]:
# Loading the model from torchaudio model hub
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()

In [4]:
# Checking the device available during the current environment (CUDA is recommended!)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [5]:
# Loading the audio
input_audio, sample_rate = torchaudio.load('/content/CRDNN_Model/AudioSamplesASR/spk1_snt1.wav')

In [6]:
# My target
target_transcription = 'THE MAN ALMOST KILL THE BIG CAMEL'
attack = ASRAttacks(model, device, bundle.get_labels())
target = list(target_transcription.upper().replace(" ", "|"))

In [7]:
#initializing attack class
attack = ASRAttacks(model, device, bundle.get_labels())

# <center>**FGSM ATTACK**</center>

```
        Simple fast gradient sign method attack is implemented which is the simplest
        adversarial attack in this domain.
        For more information, see the paper: https://arxiv.org/pdf/1412.6572.pdf

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]. 
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        targeted      : If the attack should be targeted towards your desired 
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted 
                        transcription also in this case).

        RETURNS:

        np.ndarray : Perturbed audio
```


# **FGSM TARGETED**

In [8]:
#FGSM
temp1 = attack.FGSM_ATTACK(input_audio, target, epsilon = 0.01, targeted = True)

#FGSM PRINT
print(attack.INFER(torch.from_numpy(temp1)).replace("|"," "))
print(target_transcription)

THE CHILD ALMOST HURT THE SMALL GOG 
THE MAN ALMOST KILL THE BIG CAMEL


In [9]:
Audio(temp1, rate =16000)

# **FGSM UNTARGETED**

In [10]:
#FGSM
temp2 = attack.FGSM_ATTACK(input_audio, epsilon = 0.0065, targeted = False)

#FGSM PRINT
print(attack.INFER(torch.from_numpy(temp2)).replace("|"," "))

THE CHILD ALMOST HURT  THE SMALL DOG 


In [11]:
Audio(temp2, rate =16000)

# <center>**BIM ATTACK**</center>

```
        Basic Itertive Moment attack is implemented which is simple Fast Gradient 
        Sign Attack but in loop. 
        For more information, see the paper: https://arxiv.org/abs/1607.02533

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]. 
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Noise contribution in input controlling parameter
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        targeted      : If the attack should be targeted towards your desired 
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted 
                        transcription also in this case).

        RETURNS:

        np.ndarray : Perturbed audio
```


# **BIM TARGETED**

In [12]:
#BIM
temp3 = attack.BIM_ATTACK(input_audio, target, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 3000, targeted = True)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp3)).replace("|"," "))
print(target_transcription)

  0%|          | 0/3000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !

 THE MAN ALMOST KILL THE BIG CAMEL 
THE MAN ALMOST KILL THE BIG CAMEL


In [13]:
Audio(temp3, rate = 16000)

# **BIM UNTARGETED**

In [14]:
#BIM
temp4 = attack.BIM_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 2500, targeted = False)

#BIM PRINT
print('\n',attack.INFER(torch.from_numpy(temp4)).replace("|"," "))

  0%|          | 0/2500 [00:00<?, ?it/s]

Breaking for loop because untargeted Attack is performed successfully !

 CHILD ALMOST HURT THE SMALL DOG 


In [15]:
Audio(temp4, rate =16000)

# <center>**PGD ATTACK**</center>

```
        Projected Gradient Descent attack is implemented which in simple terms is more 
        advanced version of BIM. In this attack we project our perturbation back to 
        some Lp norm and find perturbations in that particular region. 
        For more information, see the paper: https://arxiv.org/abs/1706.06083

        INPUT ARGUMENTS:

        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor

        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]. 
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also.

        epsilon       : Noise controlling parameter.
                        Type: float

        alpha         : Noise contribution in input controlling parameter
                        Type: float

        num_iter      : Number of iteration of attack
                        Type: int

        targeted      : If the attack should be targeted towards your desired 
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted 
                        transcription also in this case).

        RETURNS:

        np.ndarray : Perturbed audio
```

# **PGD TARGETED**

In [16]:
#PGD
temp5 = attack.PGD_ATTACK(input_audio, target, epsilon = 0.0015, alpha = 0.00009,
                   num_iter = 10000, targeted = True)

#PGD PRINT
print('\n',attack.INFER(torch.from_numpy(temp5)).replace("|"," "))
print(target_transcription)

  0%|          | 0/10000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !

 THE MAN ALMOST KILL THE BIG CAMEL
THE MAN ALMOST KILL THE BIG CAMEL


In [17]:
Audio(temp5, rate =16000)

# **PGD UNTARGETED**

In [18]:
#PGD
temp6 = attack.PGD_ATTACK(input_audio, epsilon = 0.0015, alpha = 0.0001,
                   num_iter = 10000, targeted = False)

#PGD PRINT
print('\n',attack.INFER(torch.from_numpy(temp6)).replace("|"," "))

  0%|          | 0/10000 [00:00<?, ?it/s]

Breaking for loop because untargeted Attack is performed successfully !

 T THE CHILD ALMOST TRRIRT  THE SNOW DOVE 


In [19]:
Audio(temp6, rate =16000)

# <center>**CW ATTACK**</center>

```
        Implements the Carlini and Wagner attack, the strongest white box 
        adversarial attack. This attack uses an optimization strategy to find the 
        adversarial transcription while keeping the perturbation as low as possible. 
        For more information, see the paper: https://arxiv.org/pdf/1801.01944.pdf
        
        INPUT ARGUMENTS:
        
        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor
                        
        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]. 
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also.
                        
        epsilon       : Noise controlling parameter.
                        Type: float
                        
        c             : Regularization term controlling factor.
                        Type: float
                        
        learning_rate : learning_rate of optimizer.
                        Type: float
                        
        num_iter      : Number of iteration of attack.
                        Type: int
                        
        decrease_factor_eps   : Factor to decrease epsilon during search
                                Type: float
                                
        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int
                                
        optimizer     : Name of the optimizer to use for the attack. 
                        Type: str
                        
        nested        : if this attack in being run in a for loop with tqdm 
                        Type: bool
                        
        early_stop    : if the user wants to end the attack as soon as the attack
                        gets the target transcription.
                        Type: bool
                        
        search_eps    : if the user wants the attack to search for the epsilon value
                        on its own provided the initial epsilon value of large.
                        Type: bool
        
        targeted      : If the attack should be targeted towards your desired 
                        transcription or not.
                        Type: bool
                        CAUTION:
                        Please make to pass your targetted 
                        transcription also in this case).
                        
        RETURNS:
        
        np.ndarray : Perturbed audio
```

# **CW TARGETED**

In [20]:
#CW
temp7 = attack.CW_ATTACK(input_audio, target, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True, 
                  early_stop = True, search_eps = False, targeted = True)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp7)).replace("|"," "))
print(target_transcription)

  0%|          | 0/10000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !

 THE MAN ALMOST KILL THE BIG CAMEL
THE MAN ALMOST KILL THE BIG CAMEL


In [21]:
Audio(temp7, rate =16000)

# **CW UNTARGETED**

In [22]:
#CW
temp8 = attack.CW_ATTACK(input_audio, epsilon = 0.0015, c = 10,
                  learning_rate = 0.00001, num_iter = 10000, decrease_factor_eps = 1,
                  num_iter_decrease_eps = 10, optimizer = None, nested = True, 
                  early_stop = True, search_eps = False, targeted = False)

#CW PRINT
print('\n',attack.INFER(torch.from_numpy(temp8)).replace("|"," "))

  0%|          | 0/10000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !

 HECHILD ALMOST HURT HE SMALL DOG 


In [23]:
Audio(temp8, rate = 16000)

# <center>**IMPERCEPTIBLE ASR ATTACK**</center>


```
        Implements the Imperceptible ASR attack, which leverages the strongest white box 
        adversarial attack which is CW attack while also masking sure the added perturbation
        is imperceptible to humans using Psychoacoustic Scale. This attack is performed in two 
        stages. In first stage we perform simple CW attack and in 2nd stage we make sure our 
        added perturbations are imperceptible. 
        For more information, see the paper: https://arxiv.org/abs/1903.10346
        
        INPUT ARGUMENTS:
        
        input__       : Input audio. Ex: Tensor[0.1,0.3,...] or (samples,)
                        Type: torch.Tensor
                        
        target        : Target transcription (needed if the you want targeted 
                        attack) Ex: ["my name is mango."]
                        Type: List[str]
                        CAUTION:
                        Please make sure these characters are also present in the 
                        dictionary of the model also
                        
        epsilon       : Noise controlling parameter
                        Type: float
                        
        c             : Regularization term controlling factor
                        Type: float
                        
        learning_rate1: learning_rate of optimizer for stage 1
                        Type: float
        
        learning_rate2: learning_rate of optimizer for stage 2
                        Type: float
                        
        num_iter1     : Number of iteration of attack stage 1
                        Type: int
        
        num_iter2     : Number of iteration of attack stage 2
                        Type: int
                        
        decrease_factor_eps   : Factor to decrease epsilon by during optimization 
                                Type: float
                                
        num_iter_decrease_eps : Number of iterations after which to decrease epsilon
                                Type: int
                                
        optimizer1     : Name of the optimizer to use for the attack stage 1
                         Type: str
                         
        optimizer2     : Name of the optimizer to use for the attack stage 2
                         Type: str
                         
        nested         : if this attack in being run in a for loop with tqdm 
                         Type: bool
        
        early_stop_cw  : if the user wants 1st stage attack to early stop or not
                         Type: bool
        
        search_eps_cw  : if the user wants 1st stage attack to search for epsilon 
                         value provided a large intial epsilon value is provided
                         Type: bool
                         
        alpha          : regularization term for second stage loss
                         Type: float
                         
        RETURNS:
        
        np.ndarray     : Perturbed audio
```

In [37]:
temp9 = attack.IMPERCEPTIBLE_ATTACK(torch.nn.functional.pad(input_audio, (0, 1000)), target, epsilon = 0.015, c = 10, learning_rate1 = 0.001, 
                                    learning_rate2 = 0.0001, num_iter1 = 1000, num_iter2 = 15000, decrease_factor_eps = 1, 
                                    num_iter_decrease_eps = 10, optimizer1 = None, optimizer2 = "Adam",nested = True, 
                                    early_stop_cw = True, search_eps_cw = False, alpha = 0.05)

# Attack's transcription
print('\n',attack.INFER(torch.from_numpy(temp9)).replace("|"," "))

***** Attack Stage 1 *****


  0%|          | 0/1000 [00:00<?, ?it/s]

Breaking for loop because targeted Attack is performed successfully !
***** Attack Stage 2 *****


  0%|          | 0/15000 [00:00<?, ?it/s]


 THE MAN ALMOST KILL THE BIG CAMEL


In [38]:
Audio(temp9[:,:-1000], rate = 16000)

----------------------------

----------------------------

# <center> **COMPARISON**

METRIC I USED FOR COMPARISON

In [39]:
# SNR metric for comparison to check which attack adds less noise
def Metricsnr(original, noisy):
    original_power = 20 * np.log10(np.mean(original ** 2))
    noise_power = 20 * np.log10(np.mean((original - noisy) ** 2))
    snr = noise_power - original_power
    return snr

In [40]:
orig = input_audio.numpy() # clean audio
orig.shape

(1, 45920)

-------------------

# **TARGETED ATTACKS**

In [41]:
Audio(temp3, rate = 16000) #BIM targeted

In [42]:
Audio(temp5, rate = 16000) #PGD targeted

In [43]:
Audio(temp7, rate = 16000) #CW targeted

In [44]:
Audio(temp9[:,:-1000], rate = 16000) # Imperceptible ASR

In [45]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR                  :",Metricsnr(orig, temp3)) 
print("PGD SNR                  :",Metricsnr(orig, temp5)) 
print("CW SNR                   :",Metricsnr(orig, temp7))
print("IMPERCEPTIBLE ATTACK SNR :",Metricsnr(orig, temp9[:,:-1000]))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR                  : -43.253135681152344
PGD SNR                  : -52.7523136138916
CW SNR                   : -58.48269462585449
IMPERCEPTIBLE ATTACK SNR : -36.3602352142334


-------------------

-------------------

# **UNTARGETED ATTACKS**

In [46]:
Audio(temp4, rate = 16000) #BIM untargeted

In [47]:
Audio(temp6, rate = 16000) #PGD untargeted

In [48]:
Audio(temp8, rate = 16000) #CW untargeted

In [49]:
print("LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED")
print("BIM SNR:",Metricsnr(orig, temp4)) 
print("PGD SNR:",Metricsnr(orig, temp6)) 
print("CW SNR :",Metricsnr(orig, temp8))

LOWER SNR IS BETTER OVER BECAUSE OF THE WAY IT IS MEASURED
BIM SNR: -76.83188438415527
PGD SNR: -52.762441635131836
CW SNR : -92.9918098449707


-------------------